In [1]:
# the library used
import numpy as np
import pandas as pd
import pysubgroup as ps
import re
from sklearn.feature_extraction import DictVectorizer
import time
import ast
import string 

## Helper Function

In [2]:
def tags_preprocess(tags):
    """
    input: tag string
    output: list of individual tags in the given tag string
    function: preprocess a single tag string 
    """
    tags = tags.replace("'","")
    tags = tags.replace(" ","")
    tags = tags.replace("[","")
    tags = tags.replace("]","")
    tags = tags.split(",")
    tags = [x.lower() for x in tags]
    return tags

In [3]:
def ingredients_preprocess(dataframe):
    """
    input: dataframe
    output: list of distinct ingredient list
    function: preprocess the ingredient columns and return a list of distinct ingredients
    """
    distinct_ingredients = []
    
    for i in range(len(dataframe)):
        ingredients = dataframe.iloc[i]['ingredient']
        
        r = re.compile('[A-Z]{1}[a-zA-Z]+')
        ingredients = str(ingredients)
        #ingredients = ''.join(i for i in ingredients if not i.isdigit())
        ingredients = ingredients.replace("'","")
        ingredients = ingredients.replace(" ","")
        ingredients = ingredients.replace("[","")
        ingredients = ingredients.replace("]","")
        # remove text inside parentheses
        ingredients = re.sub(r'\([^())]*\)',"", ingredients)
        ingredients = ingredients.split(",")
        ingredients = list(filter(r.match, ingredients))
        ingredients = [x.lower() for x in ingredients]
        distinct_ingredients += ingredients
        dataframe.iloc[i]['ingredient'] = ingredients
        
    return list(set(distinct_ingredients))

In [4]:
def get_recipe_countries(countries, data):
    """
    input: list of countries, dataframe
    output: selected dataframe whose recipes is from these countries
    function: select the rows in dataframe whose "tag" value contain one country tag
    """
    # add a new column class 
    drop_index = []
    for i in range(len(data)):
        tags = data.loc[i]["tags"]
        tags = tags_preprocess(tags)
        
        country_same =[l for l in countries if l in tags]
            
        if len(country_same) == 1:
            data.at[i, 'label'] = country_same[0]
        if len(country_same) == 0:
            drop_index.append(i)
        if len(country_same) > 1:
            drop_index.append(i)
            #data.at[i, 'label'] = 'overlap'
            
    # drop the columns which has no season tags
    data = data.drop(data.index[drop_index])
    return data

In [5]:
def convert_to_dict(arr):
    """
    Helper function to convect an array of ingredients to a dictionary
    """
    d={}
    for a in arr:
        d[a]=1
    return d

In [6]:
# extract comment user dataset from original dataset
def extract_com_user(data):
    """
    input: dataframe
    output: a new dataframe with all the comment user information
    function: spilt the dictionary of the column 'comment user' in original dataset
    """
    df_com = pd.DataFrame()
    for index, item in data['comment_user'].iteritems():
        if (item != '[]'):
            if (item != 'no comment'):
                array = ast.literal_eval(item)
                df_array = pd.DataFrame(array)
                df_array['recipe_id'] = index
                df_com = pd.concat([df_com,df_array])
    return df_com

In [7]:
def sub_cat_in_com(data):
    """
    input: dataframe
    output: a new dataframe with multi colunms
    function: add one subcategory of recipes to comment user dataset 
    """    
    punct = set(string.punctuation) 
    list_sub_cat = []
    
    df_com2 = pd.DataFrame()
    for index, item in data['calorie_value'].iteritems(): 
        if (item != None):
            list_sub = list(item)
            list_sub = ''.join(x for x in list_sub if x not in punct)
            list_sub_cat.append(list_sub)
    df_sub_cat = pd.DataFrame(list_sub_cat)
    df_sub_cat['calorie_value'] = df_sub_cat
    
    df_sub_cat['recipe_id'] = data['calorie_value'].index     
    df_com2 = pd.concat([df_com2,df_sub_cat])
    return df_com2

In [8]:
def age_group(age):   
    """
    input: age value
    output: group description
    function: divide age value uinto 5 groups 
    """   
    bucket = None
    age = int(age)    
    if age < 30:
        bucket = '<30 Jahre'    
    if age in range(30, 41):
        bucket = '30-40 Jahre'        
    if age in range(40, 51):
        bucket = '40-50 Jahre'        
    if age in range(50, 61):
        bucket = '50-60 Jahre'
    if age >= 61:
        bucket = '60+ Jahre'
    return bucket

In [9]:
def calorie_level(calorie):   
    """
    input: calorie value
    output: group description
    function: divide calorie value into 3 groups 
    """   
    bucket = None
    calorie = int(calorie)    
    if calorie < 300:
        bucket = 'low_calorie'    
    if calorie in range(300, 500):
        bucket = 'medium_calorie'        
    if calorie >= 500:
        bucket = 'high_calorie'      
    return bucket

In [10]:
def remove_None(data, name):
    """
    Helper function to remove None value in one column
    """ 
    y = data[data[name] == 'None']
    index_n = y.index.tolist()
    data = data.drop(index = index_n)
    return data

In [11]:
def add_target(data, calorie_level):
    df_sub_group[country] = df_dum_car[calorie_level]
    return df_sub_group

## Subgroup Discovery

In this section we will use subgroup discovery to explore the association rules between attributes

- why we choose subgroup discovery?

because we find out that subgroup discovery is quite powerful compared to other data mining techniques. As long as we set differnt target with different search space, we can use use subgroup discovery technique to dig almost all interesting pattern that we want to explore from the dataset. 


In [12]:
# read the data
data = pd.read_csv("all_data.csv")

In [13]:
data.head()

,categorize,recipe_name,tags,avg_score,difficulty,ingredient,rating_count,calorie,preparation_time,comment_user,recipe_url
0,Menüart,"""A bis Z""-Salat","['Salat', 'Gemüse', 'Sommer', 'Vegetarisch', '...",2.67,simpel,"['1m.-große', 'Zucchini', '1', 'Apfel,säuerlic...",1.0,NaN,15 Min,"[{'rating': 'rating-3', 'comment_time': '24.09...",https://www.chefkoch.de/rezepte/23694213760282...
1,Menüart,"""Aufgehende Sonne""","['Asien', 'Hauptspeise', 'Nudeln', 'Beilage', ...",3.60,normal,"['500g', 'Nudeln(Mie-),ReisnudelnoderReis', '4...",3.0,500kcal,30 Min,"[{'rating': 'rating-5', 'comment_time': '10.08...",https://www.chefkoch.de/rezepte/18838313064020...
2,Regional,"""Black and White"" New York Cheesecake","['Backen', 'USAoderKanada', 'Kuchen']",0.00,normal,"['250g', 'Bitterschokolade,70%', '300g', 'Schl...",0.0,NaN,60 Min,NaN,https://www.chefkoch.de/rezepte/36158315436795...
3,Menüart,"""Bleib gesund""-Smoothie SuperNova","['Getränk', 'Vegetarisch', 'Frühstück', 'kalor...",4.03,simpel,"['1m.-große', 'Karotte(n)', '1', 'Apfel', '1St...",84.0,205kcal,5 Min,"[{'rating': 'rating-1', 'comment_time': '13.01...",https://www.chefkoch.de/rezepte/20464013311985...
4,Menüart,"""Bottermelk Anballerste""","['Hauptspeise', 'Europa', 'Suppe', 'gebunden',...",3.60,normal,"['500g', 'Kartoffel(n)', '1', 'Zwiebel(n)', '1...",3.0,NaN,15 Min,"[{'rating': 'rating-3', 'comment_time': '10.11...",https://www.chefkoch.de/rezepte/33875015040182...


### Association rules between comment user information and recipe attributes

- associations with "calorie" and comment user information

- data imbalance: gender
- improve the quality of data (preprossing)
- sampling 
- improne quality of result
- if the quality is also very low, plot the t-SNE

In [15]:
pat = r"([-+]?\d*\.\d+|\d+)"
data["calorie_value"] = data["calorie"].str.extract(pat, flags=0, expand=True)
data.head()

,categorize,recipe_name,tags,avg_score,difficulty,ingredient,rating_count,calorie,preparation_time,comment_user,recipe_url,calorie_value
0,Menüart,"""A bis Z""-Salat","['Salat', 'Gemüse', 'Sommer', 'Vegetarisch', '...",2.67,simpel,"['1m.-große', 'Zucchini', '1', 'Apfel,säuerlic...",1.0,NaN,15 Min,"[{'rating': 'rating-3', 'comment_time': '24.09...",https://www.chefkoch.de/rezepte/23694213760282...,NaN
1,Menüart,"""Aufgehende Sonne""","['Asien', 'Hauptspeise', 'Nudeln', 'Beilage', ...",3.60,normal,"['500g', 'Nudeln(Mie-),ReisnudelnoderReis', '4...",3.0,500kcal,30 Min,"[{'rating': 'rating-5', 'comment_time': '10.08...",https://www.chefkoch.de/rezepte/18838313064020...,500
2,Regional,"""Black and White"" New York Cheesecake","['Backen', 'USAoderKanada', 'Kuchen']",0.00,normal,"['250g', 'Bitterschokolade,70%', '300g', 'Schl...",0.0,NaN,60 Min,NaN,https://www.chefkoch.de/rezepte/36158315436795...,NaN
3,Menüart,"""Bleib gesund""-Smoothie SuperNova","['Getränk', 'Vegetarisch', 'Frühstück', 'kalor...",4.03,simpel,"['1m.-große', 'Karotte(n)', '1', 'Apfel', '1St...",84.0,205kcal,5 Min,"[{'rating': 'rating-1', 'comment_time': '13.01...",https://www.chefkoch.de/rezepte/20464013311985...,205
4,Menüart,"""Bottermelk Anballerste""","['Hauptspeise', 'Europa', 'Suppe', 'gebunden',...",3.60,normal,"['500g', 'Kartoffel(n)', '1', 'Zwiebel(n)', '1...",3.0,NaN,15 Min,"[{'rating': 'rating-3', 'comment_time': '10.11...",https://www.chefkoch.de/rezepte/33875015040182...,NaN


In [57]:
data_com = data.dropna(subset=["comment_user",'calorie_value'])
# data_com = data_com[0:1800]
data_com.head()
# data_com data_com_com['comment_user']
df_com_1 = extract_com_user(data_com)
df_com_2 = sub_cat_in_com(data_com)
df_com_new = df_com_1.merge(df_com_2, on='recipe_id', how='left')

df_com_new = df_com_new.drop(0,1)
df_com_new = df_com_new[['recipe_id','name','rating','sex','age','marriage_status','job','comment_time','calorie_value']]
df_com_new = df_com_new.set_index(["recipe_id"])
df_com_new
# df_com_new['calorie_value'].value_counts()

,name,rating,sex,age,marriage_status,job,comment_time,calorie_value
recipe_id,,,,,,,,
1,Haubndaucher,rating-5,männlich,46 Jahre,Verheiratet,Verkäufer.,10.08.2017 14:56,500
1,Lumakath,rating-4,weiblich,49 Jahre,Verheiratet,None,19.01.2013 13:19,500
1,esther76,rating-4,weiblich,None,Verheiratet,Busfahrerin,21.10.2012 17:56,500
3,t1n4a,rating-1,weiblich,25 Jahre,None,None,13.01.2020 21:05,205
3,patty89,rating-4,weiblich,None,Verheiratet,None,03.09.2019 13:18,205
...,...,...,...,...,...,...,...,...
99286,tig,rating-5,weiblich,None,None,None,22.12.2017 16:42,493
99286,Kuchlmasta,rating-5,männlich,65 Jahre,None,Pension,18.07.2016 18:47,493
99286,Fee56,rating-5,None,None,None,Pflegefachkraft,18.07.2016 11:09,493


In [43]:
df_com_new['calorie_level'] = df_com_new['calorie_value'].apply(calorie_level)
# df_dum_car = pd.get_dummies(df_com_new['calorie_level'])
# df_com_car = pd.merge(df_com_new,df_dum_car,left_on='name', right_on=None, how='left')
# df_com_car = pd.merge(df_com_new, df_dum_car)
# df_com_car = df_com_new.join(df_dum_car, how='left')
df_dum_car


,high_calorie,low_calorie,medium_calorie
recipe_id,,,
43,1,0,0
43,1,0,0
61,1,0,0
79,0,0,1
79,0,0,1
...,...,...,...
8471,1,0,0
8471,1,0,0
8471,1,0,0


In [44]:
# add age group column
df_com_age = remove_None(df_com_new, 'age')
df_com_age["age_value"] = df_com_age["age"].str.extract(pat, flags=0, expand=True)
df_com_age['age_group'] = df_com_age['age_value'].apply(age_group)
df_dum_car = pd.get_dummies(df_com_age['calorie_level'])
df_age = df_com_age[['rating','sex','age_group']]
df_age['high_calorie'] = df_dum_car['high_calorie']
df_age['low_calorie'] = df_dum_car['low_calorie']
df_age['medium_calorie'] = df_dum_car['medium_calorie']
df_age

/Users/sunwen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/sunwen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/sunwen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a cop

,rating,sex,age_group,high_calorie,low_calorie,medium_calorie
recipe_id,,,,,,
43,rating-4,weiblich,30-40 Jahre,1,0,0
43,rating-5,weiblich,50-60 Jahre,1,0,0
61,rating-5,weiblich,40-50 Jahre,1,0,0
79,rating-4,weiblich,50-60 Jahre,0,0,1
79,rating-4,weiblich,<30 Jahre,0,0,1
...,...,...,...,...,...,...
8471,rating-5,männlich,<30 Jahre,1,0,0
8471,rating-5,weiblich,40-50 Jahre,1,0,0
8471,rating-3,weiblich,30-40 Jahre,1,0,0


In [55]:
df_age['age_group'].value_counts()
# df_age['medium_calorie'].value_counts()


30-40 Jahre    122
50-60 Jahre     88
40-50 Jahre     85
<30 Jahre       47
60+ Jahre       41
Name: age_group, dtype: int64

In [53]:
import pysubgroup as ps


target = ps.BinaryTarget ('medium_calorie', True)
searchspace = ps.create_selectors(df_age, ignore=['high_calorie','medium_calorie','low_calorie','rating'])
task = ps.SubgroupDiscoveryTask (
    df_age, 
    target, 
    searchspace, 
    result_set_size=5, 
    depth=3, 
    qf=ps.WRAccQF())
result = ps.BeamSearch().execute(task)
print(result.to_dataframe())

    quality                                   description
0  0.009537                        age_group=='60+ Jahre'
1  0.008044    age_group=='60+ Jahre' AND sex=='weiblich'
2  0.007744  age_group=='50-60 Jahre' AND sex=='weiblich'
3  0.006551  age_group=='40-50 Jahre' AND sex=='männlich'
4  0.006531                               sex=='männlich'


### Association rules among the recipe attributes

- the association with "ingredient" and "regions"

In [ ]:
countries = ['china', 'indien', 'deutschland','frankreich','grossbritannien','österreich','usaoderkanada','italien','spanien',
             'portugal', 'japen','schweiz','türkei', 'thailand', 'russland', 'großbritannien & irland', 'vietnam', 'korea',
            'australien', 'ägypten', 'marokko', 'niederlande']

In [ ]:
df_certain_countries = get_recipe_countries(countries, data)
df_certain_countries.reset_index(inplace = True)
df_certain_countries['label'].value_counts()

In [ ]:
# drop all the columns other than "ingredient" and "label" column
columns_drop = ['index', 'categorize', 'recipe_name', 'tags', 'avg_score', 'difficulty','rating_count', 'calorie', 
                'preparation_time','comment_user', 'recipe_url']
df_certain_countries = df_certain_countries.drop(columns_drop, axis = 1)

# preprocess the ingredient column
distinct_ingredients = ingredients_preprocess(df_certain_countries)

# One hot encoding of the ingredients
df_certain_countries['ingredient'] = df_certain_countries.ingredient.apply(convert_to_dict)
vectorizer = DictVectorizer(sparse=False)
new_df_countries = pd.DataFrame(data = vectorizer.fit_transform(df_certain_countries.ingredient.tolist()), columns = distinct_ingredients)
new_df_countries['label'] = df_certain_countries.label

# dummy for the label column
new_df_countries = pd.get_dummies(new_df_countries)

In [ ]:
# apply 
new_df_countries

In [ ]:
test = new_df_countries.iloc[:,]
test = test.iloc[:4]

In [ ]:
country_label = list(new_df_countries.iloc[:,-19:].columns)
#contry_label.remove('label_deutschland')

# record the start time
time1 = time.time()
        
# apply subgroup discovery 
target = ps.BinaryTarget ('label_usaoderkanada', True)
searchspace = ps.create_selectors(test, ignore = country_label)
task = ps.SubgroupDiscoveryTask (
    data, 
    target, 
    searchspace, 
    result_set_size=5, 
    depth=20, 
    qf=ps.WRAccQF())
result = ps.BeamSearch().execute(task)

# record the end time
time2 = time.time()
time_diff = (time2-time1)/60
print('it took ' + str(time_diff) + 'miniutes to execute the subgroup disc')